In [1]:
# dependencies
import tensorflow as tf
import numpy as np
from DQN import DQN
from Experience import ExperienceBuffer
from Game import GameEnv

In [2]:
# opts
load_model = False
checkpoints_path = 'checkpoints/'
# choose one from checkpoints path
model_path = checkpoints_path + '-500'
save_period = 500
info_period = 10
update_period = 10

In [3]:
# hyperparameters
batch_size = 50
gamma = 0.9
s_epsilon = 0.9
e_epsilon = 0.1
epsilon = s_epsilon
drop_step = (s_epsilon - e_epsilon) / 5000
num_episodes = 10000
pre_train = 5000
max_ep_len = 10000

In [4]:
# create game env, model and exp buff
env = GameEnv()
dqn = DQN()
experience_buffer = ExperienceBuffer(30000)

In [5]:
# training
total_steps = 0
init = tf.global_variables_initializer()
saver = tf.train.Saver()
rewards = []

with tf.Session() as sess:
    try:
        if load_model:
            saver.restore(sess, model_path)
        else:
            sess.run(init)

        for i in range(num_episodes):
            episode_buffer = ExperienceBuffer(10000)
            episode_reward = 0.0

            s, r, d = env.reset(i==0, i>0)

            for j in range(max_ep_len):
                if np.random.rand(1) < epsilon or total_steps < pre_train:
                    a = np.random.randint(0, 3)
                else:
                    a = sess.run(dqn.predict, feed_dict={dqn.x: [s]})[0]

                s1, r, d = env.step(a)
                episode_buffer.add(np.reshape(np.array([s, a, r, s1, d]), [1,5]))
                episode_reward += r
                s = s1
                total_steps += 1

                if total_steps > pre_train:
                    if epsilon > e_epsilon:
                        epsilon -= drop_step

                    if total_steps % update_period == 0:
                        
                        batch = experience_buffer.batch(batch_size)
                        Q1 = sess.run(tf.reduce_max(dqn.q_vals, axis=1), feed_dict={dqn.x: np.vstack(batch[:, 3])})
                        end_mul = -(batch[:, 4] - 1)
                        targetQ = batch[:, 2] + gamma * Q1 * end_mul

                        sess.run(dqn.train, feed_dict={dqn.x: np.vstack(batch[:, 0]), 
                                                       dqn.targetQ: targetQ,
                                                       dqn.actions: batch[:, 1]})

                if d:
                    break
            experience_buffer.add(episode_buffer._buffer)
            rewards.append(episode_reward)

            if i % save_period == 0:
                print 'model saved'
                saver.save(sess, checkpoints_path, global_step=i)

            if i % info_period == 0:
                print '|------------------------|'
                print ('total episodes: {}\n'
                       'total steps: {}\n'
                       'last {} episodes average reward: {}'.format(i+1, total_steps, info_period, 
                                                                    np.mean(rewards[-info_period:])))
                print '|------------------------|'
    except KeyboardInterrupt:
        env.exit()
        print 'Training stopped'

env.exit()
print 'Finished'

10 seconds before start, focus tab with game!
model saved
|------------------------|
total episodes: 1
total steps: 59
last 10 episodes average reward: 58.0
|------------------------|
|------------------------|
total episodes: 11
total steps: 685
last 10 episodes average reward: 61.6
|------------------------|
|------------------------|
total episodes: 21
total steps: 1280
last 10 episodes average reward: 58.5
|------------------------|
|------------------------|
total episodes: 31
total steps: 1845
last 10 episodes average reward: 55.5
|------------------------|
|------------------------|
total episodes: 41
total steps: 2411
last 10 episodes average reward: 55.6
|------------------------|
|------------------------|
total episodes: 51
total steps: 2972
last 10 episodes average reward: 55.1
|------------------------|
|------------------------|
total episodes: 61
total steps: 3517
last 10 episodes average reward: 53.5
|------------------------|
|------------------------|
total episodes: 

OSError: [Errno 12] Cannot allocate memory